# OPAN5505 Lab 3 Assignment

## Background

You've been engaged by the Federal Aviation Administration to perform an analysis on "strike" data. A "strike" refers to when an aircraft and an object within the air (typically wildlife) collide.

## Prepare your environment

You will need the `polars` package for this assignment. We'll also use `datetime` for date handling.

In [9]:
# Import required libraries

# Import necessary libraries
import polars as pl
from datetime import datetime

# Optional: Configure Polars display settings
pl.Config.set_tbl_rows(10)   # Display up to 10 rows
pl.Config.set_tbl_cols(10)   # Display up to 10 columns

# Verify that everything is set up correctly
print("Polars version:", pl.__version__)
print("Current datetime:", datetime.now())



Polars version: 1.25.2
Current datetime: 2025-08-17 20:29:07.631118


## Load your dataset

Read the `faa_strikes.txt` dataset into Polars. Name the resulting DataFrame `strikes`.

In [14]:
# Read the faa_strikes.txt data


import polars as pl

# Load the dataset
strikes = pl.read_csv("faa_strikes.txt", separator="\t")

# Show the first few rows to confirm it's loaded
print(strikes.head())


shape: (5, 25)
┌─────────┬─────────┬─────────┬─────────┬─────────┬───┬────────┬────────┬────────┬────────┬────────┐
│ Airport ┆ Airport ┆ Origin  ┆ Origin  ┆ Country ┆ … ┆ Wildli ┆ Wildli ┆ Wildli ┆ Number ┆ Record │
│ : Code  ┆ : Name  ┆ State   ┆ State   ┆ ---     ┆   ┆ fe:    ┆ fe:    ┆ fe:    ┆ of Str ┆ ID     │
│ ---     ┆ ---     ┆ ---     ┆ Code    ┆ str     ┆   ┆ Specie ┆ Specie ┆ Specie ┆ ikes   ┆ ---    │
│ str     ┆ str     ┆ str     ┆ ---     ┆         ┆   ┆ s      ┆ s      ┆ s ID   ┆ ---    ┆ i64    │
│         ┆         ┆         ┆ str     ┆         ┆   ┆ Group  ┆ ---    ┆ ---    ┆ i64    ┆        │
│         ┆         ┆         ┆         ┆         ┆   ┆ ---    ┆ str    ┆ str    ┆        ┆        │
│         ┆         ┆         ┆         ┆         ┆   ┆ str    ┆        ┆        ┆        ┆        │
╞═════════╪═════════╪═════════╪═════════╪═════════╪═══╪════════╪════════╪════════╪════════╪════════╡
│ KAAF    ┆ APALACH ┆ Florida ┆ FL      ┆ United  ┆ … ┆ Wolves ┆ Domest ┆ 1F

## Question 1: Running total of strikes by day through 2013

You are interested in seeing the running total of strikes over time (`Collision Date and Time`) on a daily basis. Using the `strikes` dataframe, create a new column that removes the time information but keeps the date information from the `Collision Date and Time` column (call this new field `date`). Next, aggregate the `Number of Strikes` by day (call this new field `daily_strikes`). Sort the data in ascending order by `date`. Then, create the running total of `daily_strikes` (name the new field `strikes_cumulative`) and filter to only the records **up to and including** the `date` of 2013-12-31. Name the resulting data frame `running_total_strikes`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [59]:
# Your code here

import pandas as pd

# Load the dataset
strikes = pd.read_csv("faa_strikes.txt", sep="\t")

# Convert to datetime
strikes["Collision Date and Time"] = pd.to_datetime(
    strikes["Collision Date and Time"], errors="coerce"
)

# Drop invalid datetimes
strikes = strikes.dropna(subset=["Collision Date and Time"])

# Extract just the date
strikes["date"] = strikes["Collision Date and Time"].dt.date

# Aggregate by day
daily_strikes = (
    strikes.groupby("date", as_index=False)["Number of Strikes"]
    .sum()
    .rename(columns={"Number of Strikes": "daily_strikes"})
    .sort_values("date")
)

# Use a window function for the cumulative sum
daily_strikes["strikes_cumulative"] = (
    daily_strikes["daily_strikes"]
    .expanding()
    .sum()
)

# Filter to only dates up to and including 2013-12-31
running_total_strikes = daily_strikes[
    daily_strikes["date"] <= pd.to_datetime("2013-12-31").date()
]

# Show the result
print(running_total_strikes)

            date  daily_strikes  strikes_cumulative
0     2000-01-02              1                 1.0
1     2000-01-03              2                 3.0
2     2000-01-05              3                 6.0
3     2000-01-06              1                 7.0
4     2000-01-08              4                11.0
...          ...            ...                 ...
4771  2013-12-27              6             24096.0
4772  2013-12-28              1             24097.0
4773  2013-12-29              2             24099.0
4774  2013-12-30              4             24103.0
4775  2013-12-31              6             24109.0

[4776 rows x 3 columns]


## Question 2: States with the third highest financial cost

The FAA is interested in the financial cost of strikes for each `Origin State`. Using the `strikes` dataframe: first, sum `Cost: Total $` by `Origin State` (name the resulting column `damage`) then `rank` these states in order of which state had the most financial `damage` (name the new column with rank information, `ranking`). Next, filter to show the row with the third highest `ranking`. Use a window function to answer this question. Name the resulting dataframe `damage_state`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [63]:
# Your code here

import pandas as pd

# Step 1: Load the dataset
strikes = pd.read_csv("faa_strikes.txt", sep="\t")

# Step 2: Convert cost column to numeric
strikes["Cost: Total $"] = pd.to_numeric(strikes["Cost: Total $"], errors="coerce")

# Step 3: Group by Origin State and sum damage
damage_by_state = (
    strikes.groupby("Origin State", as_index=False)["Cost: Total $"]
    .sum()
    .rename(columns={"Cost: Total $": "damage"})
)

# Step 4: Rank all states using a window function (dense rank)
damage_by_state["ranking"] = damage_by_state["damage"].rank(method="dense", ascending=False)

# Step 5: Filter to states with rank = 3
damage_state = damage_by_state[damage_by_state["ranking"] == 3].reset_index(drop=True)

# Final output as a DataFrame
damage_state



,Origin State,damage,ranking
0,California,29671432,3.0


## Question 3: What are the second costliest Species Groups for each Aircraft Type?

The FAA wants to know if some species groups are more dangerous to particular types of aircraft. Using the `strikes` dataframe: first, sum financial damage (`Cost: Total $`) information by `Aircraft: Type` and `Wildlife: Species Group` (name the new field `damage`). Rank the rows within each `Aircraft: Type` based on the greatest amount of `damage`; when performing the ranking function, name the new column `ranking`. Return the rows that represent the `Wildlife: Species Group` values that caused the **second most** financial `damage` to each `Aircraft: Type`. Name the resulting dataframe `type_species`.

*Please place a Python code block and code below. The output should be a DataFrame. You must use a ranking function to receive credit on this question.*

In [65]:
# Your code here

import pandas as pd

# Step 1: Load the dataset
strikes = pd.read_csv("faa_strikes.txt", sep="\t")

# Step 2: Convert financial cost to numeric
strikes["Cost: Total $"] = pd.to_numeric(strikes["Cost: Total $"], errors="coerce")

# Step 3: Sum damage by Aircraft Type and Species Group
damage_by_type_species = (
    strikes
    .groupby(["Aircraft: Type", "Wildlife: Species Group"], as_index=False)["Cost: Total $"]
    .sum()
    .rename(columns={"Cost: Total $": "damage"})
)

# Step 4: Rank within each Aircraft Type using a window function
damage_by_type_species["ranking"] = (
    damage_by_type_species
    .groupby("Aircraft: Type")["damage"]
    .rank(method="dense", ascending=False)
)

# Step 5: Filter for second-highest damage (rank = 2)
type_species = damage_by_type_species[damage_by_type_species["ranking"] == 2].reset_index(drop=True)

# Output: type_species DataFrame with the results
type_species



,Aircraft: Type,Wildlife: Species Group,damage,ranking
0,Airplane,"Kites, Hawks, Eagles",48989287,2.0
1,Helicopter,Vultures,116748,2.0


## Question 4: Which days had the greatest positive jump in strikes?

The FAA wants to investigate which days had the largest increase in strikes from the previous day. Using the `strikes` dataframe: sum the `Number of Strikes` measure by day (note: not by day and time). You can use a similar technique from Question 1 to create a new column from `Collision Date and Time` without time information and with date information (name this column `date`). Next, compute the previous day's strikes using a window function (name the new column `previous_day`), and calculate the difference of strikes between the current day and the previous day (name the new column `delta_strikes`). Sort the resulting data by `delta_strikes` in descending order to find which days had the highest increase of strikes from the previous day. Name the resulting dataframe `greatest_strike_increase`.

**Hint:** This code is more involved that other exercises, so I will explicitly name the steps to answer the question.

1. Start with the `strikes` data frame
2. Create a new column called `date` which removes the time information in the `Collision Date and Time` column
3. Sum the `Number of Strikes` by `date`, and name the new column `daily_strikes`
4. Sort by the `date` column in ascending order
5. Create a new column called `previous_day` to calculate what the `daily_strikes` were in the previous day
6. Create a new column called `delta_strikes` which subtracts `previous_day` from `daily_strikes`
7. Sort in descending order by `delta_strikes`

*Please place a Python code block and code below. The output should be a DataFrame. You must use a window function to receive credit on this question.*

In [67]:
# Your code here

import pandas as pd

# Step 1: Load the dataset
strikes = pd.read_csv("faa_strikes.txt", sep="\t")

# Step 2: Convert to datetime
strikes["Collision Date and Time"] = pd.to_datetime(strikes["Collision Date and Time"], errors="coerce")

# Step 3: Create a new 'date' column
strikes["date"] = strikes["Collision Date and Time"].dt.date

# Step 4: Sum Number of Strikes by date
daily = (
    strikes.groupby("date", as_index=False)["Number of Strikes"]
    .sum()
    .rename(columns={"Number of Strikes": "daily_strikes"})
    .sort_values("date")  # Make sure it's in chronological order
)

# Step 5: Use window function to get previous day's value
daily["previous_day"] = daily["daily_strikes"].shift(1)  # window function: shift

# Step 6: Compute the change in strikes
daily["delta_strikes"] = daily["daily_strikes"] - daily["previous_day"]

# Step 7: Sort by delta_strikes descending
greatest_strike_increase = daily.sort_values("delta_strikes", ascending=False).reset_index(drop=True)

# Final output
greatest_strike_increase

,date,daily_strikes,previous_day,delta_strikes
0,2010-10-29,25,7.0,18.0
1,2014-08-25,25,8.0,17.0
2,2012-06-28,21,5.0,16.0
3,2009-07-07,21,7.0,14.0
4,2013-11-04,22,8.0,14.0
...,...,...,...,...
5280,2012-06-29,6,21.0,-15.0
5281,2007-09-11,1,16.0,-15.0
5282,2014-10-29,5,24.0,-19.0
5283,2013-08-20,8,28.0,-20.0


## Question 5: Which single day had the greatest increase in strikes for each `Aircraft: Type`?

FAA was interested in the exercise from the last question but now wants to determine the largest delta between days for each `Aircraft: Type`. What are the days that had largest positive change in strikes for each `Aircraft: Type`? In order to answer this question, you will need to perform the exercise from Question 4, but this time include a grouping by `Aircraft: Type`. After grouping by `Aircraft: Type`, add a column called `ranking` and use a window function to determine the day with the largest increase in strikes from the previous day. Name the resulting data frame `greatest_strike_increase_type`

NOTE: Helicopters are not struck with a high frequency and will not show up in your analysis; this is fine.

In [70]:
 # Your code here
import pandas as pd

# Step 1: Load the dataset
strikes = pd.read_csv("faa_strikes.txt", sep="\t")

# Step 2: Convert Collision Date and Time to datetime
strikes["Collision Date and Time"] = pd.to_datetime(
    strikes["Collision Date and Time"], errors="coerce"
)

# Step 3: Remove rows with missing Aircraft: Type
strikes = strikes.dropna(subset=["Aircraft: Type"])

# Step 4: Extract date only
strikes["date"] = strikes["Collision Date and Time"].dt.date

# Step 5: Group by Aircraft Type and Date, and sum strikes
daily_by_type = (
    strikes.groupby(["Aircraft: Type", "date"], as_index=False)["Number of Strikes"]
    .sum()
    .rename(columns={"Number of Strikes": "daily_strikes"})
    .sort_values(["Aircraft: Type", "date"])
)

# Step 6: Get previous day's strikes per Aircraft Type using a window function
daily_by_type["previous_day"] = (
    daily_by_type.groupby("Aircraft: Type")["daily_strikes"].shift(1)
)

# Step 7: Compute change (delta) in strikes
daily_by_type["delta_strikes"] = (
    daily_by_type["daily_strikes"] - daily_by_type["previous_day"]
)

# Step 8: Rank within each Aircraft Type by delta_strikes
daily_by_type["ranking"] = (
    daily_by_type.groupby("Aircraft: Type")["delta_strikes"]
    .rank(method="dense", ascending=False)
)

# Step 9: Filter to rows with rank 1 (largest increase per type)
greatest_strike_increase_type = (
    daily_by_type[daily_by_type["ranking"] == 1].reset_index(drop=True)
)

#Final output
greatest_strike_increase_type



,Aircraft: Type,date,daily_strikes,previous_day,delta_strikes,ranking
0,Airplane,2014-08-25,25,8.0,17.0,1.0
1,Helicopter,2010-07-21,2,1.0,1.0,1.0
2,Helicopter,2012-11-02,2,1.0,1.0,1.0
3,Helicopter,2013-10-28,2,1.0,1.0,1.0
4,Helicopter,2014-08-14,2,1.0,1.0,1.0
